# Azure Machine Learning - Model Package Building (Local)
Sample notebook demonstrating how to pull an image from a target registry, run locally as web service, and submitting data to the model for scoring via a HTTP request

### Import required packages

In [ ]:
from azureml.core import Workspace, Model, Dataset
import subprocess
import requests
import json
import warnings
warnings.filterwarnings('ignore')

### Get connection to AML workspace and default Key Vault

In [ ]:
ws = Workspace.from_config()
kv = ws.get_default_keyvault()

### Retrieve ACR credentials from key vault
<i>Note:</i> If these credentials have not been previously set you will need to add them using the syntax below:
```
kv.set_secret('acr-username', '<YOUR-ACR-USERNAME>')
kv.set_secret('acr-password', '<YOUR-ACR-PASSWORD>')
kv.set_secret('acr-address', '<YOUR-ACR-ADDRESS>')
```

In [ ]:
acr_username = kv.get_secret('acr-username')
acr_password = kv.get_secret('acr-password')
acr_address = kv.get_secret('acr-address')

### Retreive model and define InferenceConfig

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

model = Model(ws, 'iris-classification')
model_name=model.name
model_version = model.version

env = Environment.get(ws, 'sample_env')

inference_config = InferenceConfig(
    environment=env,
    source_directory="./deployment",
    entry_script="score.py",
)

### Create model package and save locally

In [ ]:
package = Model.package(ws, [model], inference_config, image_name=model_name, image_label=model_version, generate_dockerfile=True)

import os
import shutil
try:
    shutil.rmtree('./model_package')
except Exception:
    pass
os.makedirs('./model_package', exist_ok=True)
package.save('./model_package')

### Update Dockerfile in locally-saved model package
Code snippet below inserts lines under `new_lines` into AML-generated Dockerfile

In [ ]:
# Update Dockerfile
lines = []
with open('./model_package/Dockerfile', 'r') as file:
    lines = file.readlines()
    
new_lines = [
    'RUN apt-get update -y\n',
    'RUN apt-get install openssl -y\n',
    'RUN pip3 install pandas==1.0.4 joblib==1.2.0 Werkzeug==2.1.1\n',
    'RUN apt-get clean -y\n'
    'RUN rm /etc/ssh/ssh_host_ecdsa_key\n'
    'RUN rm /etc/ssh/ssh_host_ed25519_key\n'
    'RUN rm /etc/ssh/ssh_host_rsa_key\n'
    'RUN rm /var/lib/apt/lists/packages.microsoft.com_ubuntu_20.04_prod_dists_focal_InRelease\n'
    'RUN rm /azureml-envs/*/lib/python3.8/site-packages/ndg/httpsclient/test/pki/localhost.key\n'
]
lines = lines[0:-1] + new_lines + [lines[-1]]

with open('./model_package/Dockerfile', 'w') as file:
    for line in lines:
        file.write(line)

### Build docker image from model package definition

In [ ]:
import docker
import datetime
kv = ws.get_default_keyvault()
acr_username = kv.get_secret('acr-username')
acr_password = kv.get_secret('acr-password')
acr_address = kv.get_secret('acr-address')
timestamp = datetime.datetime.now().strftime("%m%d%H%M%f")
location = f'{acr_address}/{model_name}:{timestamp}'
location
client = docker.from_env()
client.login(username=acr_username, password=acr_password, registry=acr_address)

# Pull existing model image
img = client.images.build(path='./model_package', tag=location, quiet=False)

print(client.images.list())

### Run docker container locally

In [ ]:
import subprocess
subprocess.check_call(['docker', 'run', '-d', '-p', '5001:5001', location])

### Retrieve testing data and test endpoint locally before pushing to ACR

In [ ]:
dataset = Dataset.get_by_name(ws, 'Testing_Data')
df = dataset.to_pandas_dataframe()
df.drop(columns=['target'], inplace=True)
records = df.to_json(orient='records')

import time
time.sleep(5)

uri = 'http://localhost:5001/score'
headers = {'Content-Type': 'application/json'}
response = requests.post(uri, data=records, headers=headers)
response.json()

### If sample HTTP request is successful push container to ACR, else cancel gracefully

In [ ]:
if response.status_code==200:
    client.images.push(location)
else:
    # run.cancel()
    pass